# Spark JDBC

En el corazón de la integración de Spark con bases de datos relacionales encontramos JDBC (Java Database Connectivity). JDBC actúa como un puente esencial, proporcionando una interfaz estandarizada que permite a las aplicaciones Spark comunicarse con bases de datos relacionales. Esta interfaz no es simplemente un canal de comunicación; es un conjunto completo de protocolos y estándares que facilitan operaciones de lectura y escritura de manera eficiente y segura.

## Spark SQL y su Relación con JDBC

Spark SQL emerge como uno de los módulos más poderosos dentro del ecosistema Spark. Este módulo introduce el concepto de DataFrames, una abstracción que permite trabajar con datos estructurados de manera intuitiva y eficiente. Cuando combinamos Spark SQL con JDBC, obtenemos una herramienta extremadamente versátil para el procesamiento de datos.  

La integración de Spark SQL con JDBC va más allá de simples operaciones de lectura y escritura. El sistema permite ejecutar consultas complejas que se benefician del procesamiento distribuido de Spark, mientras mantiene la integridad y las características ACID de las bases de datos relacionales. Esta simbiosis permite aprovechar lo mejor de ambos mundos: la escalabilidad de Spark y la confiabilidad de las bases de datos relacionales.

## Arquitectura

La arquitectura de la integración Spark-JDBC se construye sobre varios componentes fundamentales que trabajan en conjunto. El Driver JDBC actúa como el intérprete principal, traduciendo las instrucciones de Spark en comandos que la base de datos puede entender. Este componente maneja no solo la traducción de comandos, sino también la gestión de tipos de datos y la optimización de consultas.  

El Connection Pool representa otro componente crucial en esta arquitectura. En lugar de crear nuevas conexiones para cada operación, mantiene un conjunto de conexiones activas que pueden ser reutilizadas. Este enfoque reduce significativamente la sobrecarga asociada con el establecimiento de conexiones y mejora el rendimiento general del sistema.  

El sistema de particionamiento en esta arquitectura merece especial atención. Permite dividir grandes conjuntos de datos en fragmentos manejables que pueden procesarse en paralelo. Este particionamiento no es arbitrario; se basa en estrategias sofisticadas que consideran la distribución de datos y los recursos disponibles.

## Operaciones y Optimización

Las operaciones en el contexto de Spark-JDBC pueden clasificarse en tres categorías principales: lectura, escritura y transformación. Las operaciones de lectura pueden variar desde la simple recuperación de tablas completas hasta consultas complejas con múltiples joins y agregaciones. La escritura, por otro lado, puede implicar inserciones masivas, actualizaciones o operaciones de upsert.  

La optimización en este contexto es un arte complejo. El push-down de predicados representa una de las técnicas más importantes, permitiendo que los filtros se ejecuten en la base de datos antes de que los datos se transfieran a Spark. Esto puede reducir significativamente la cantidad de datos transferidos y mejorar el rendimiento general.  

La gestión de recursos y la configuración de parámetros juegan un papel crucial en el rendimiento. El tamaño del fetch, el número de particiones y el tamaño del batch deben ajustarse cuidadosamente según las características específicas de cada caso de uso. Estos ajustes pueden tener un impacto significativo en el rendimiento y la utilización de recursos.

## Ejemplo de conexión de JDBC desde Databricks Community con SQL Server de Azure. 

### Creación de SQL Server en Azure  

- Crear un grupo de recursos. Luego crear un recurso > Bases de datos > **SQL Database**  
- Configura los detalles:  
  -. Nombre del sesrvidor SQL: databricks-sql-server  
  -. Región: Italy North u otra disponible  
  -. Autenticación: Habilita SQL Authentication y configura:  
    1. Usuario: `adminuser`  
    2. Contraseña: `ContraseñaFuerte123`  
- Marca la casilla de "Habilitar acceso a Azure Services"  
  

### Crear la base de datos: AdventureWorksLT  

- En la misma sección, crea una base de datos con nombre cualquiera (en este ejemplo le hemos llamado db-notebook-4). Selecciona el servidor creado (databricks-sql-server).Nivel de precio: Elige el plan más económico (Básico - DTU: 5).   
- Ve al servidor que has creado y en networking configurar el firewall. Añade la IP pública de tu conexión local y habilita "Permitir acceso a todos los servicios de Azure". Habilita la dirección IP publica de databricks, suele ser: 54.200.13.2. Guarda los cambios.  
- En Additional settings, en Data Source escoger Sample para que se habilite la carga de AdventureWorksLT. Lo demas se deja por defecto.  


Importante: Para saber que dirección IP vamos a configurar en las reglas de firewall usar este comando:

In [0]:
import requests
 
# Obtener la IP pública del nodo
public_ip = requests.get('https://api.ipify.org').text
print(f"La IP pública del nodo es: {public_ip}")

La IP pública del nodo es: 34.214.70.55


### Configurar Databricks Community Edition  
Ve a la pestaña Computey selecciona Create Compute con estos requisitos:  
  - Cluster Name: AdventureWorksCluster.  
  - Databricks Runtime Version: 11.3 LTS (Scala 2.12, Spark 3.3.1)  
  - Crear clúster.  
  - Mientras el clúster se esta creando, descarga el controlador JDBC para SQL Server, en este caso usaremos [este](https://tajamar365.sharepoint.com/:u:/s/3405-MasterIA2024-2025/EeR4l4udCBFClfiDcBhI7PMBG-VfNpuLHlAQ7a1FVBC5OA?e=Igqclx).  
  - En Databricks subir el controlador a tu workspace o a tu DBFS.
  - Una vez que el cluster esté activo ve al Cluster y en el boton Libraries cargar el controlador haciendo click en `Install New` y le pasas el path donde has guardado el controlador.  

### Conectar Databricks con SQL Server  
Crea un notebook en Databricks y añade el siguiente código ( con tus datos de configuración) :

In [0]:
# Configuración de conexión JDBC
jdbcHostname = "databricks-sql-server-leo.database.windows.net" # Servidor SQL
jdbcPort = 1433
jdbcDatabase = "database-sql" # Nombre exacto de tu base de datos
jdbcUsername = "adminuser" # Cambiar por tu usuario configurado
jdbcPassword = "ContraseñaFuerte123" # Cambiar por la contraseña configurada

jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};databaseName={jdbcDatabase}"

# Propiedades de conexión
connectionProperties = {
  "user": jdbcUsername,
  "password": jdbcPassword,
  "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

### Consulta de Prueba

In [0]:
# Consulta de prueba
query = "(SELECT TOP 10 * FROM SalesLT.Product) AS temp"  # Cambia por una tabla válida si es necesario

# Leer datos desde SQL Server
try:
    df = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)
    df.show()  # Mostrar los datos
except Exception as e:
    print(f"Error al conectar: {e}")

+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------------+-------------------+----------------+--------------------+----------------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|Color|StandardCost|ListPrice|Size| Weight|ProductCategoryID|ProductModelID|      SellStartDate|        SellEndDate|DiscontinuedDate|      ThumbNailPhoto|ThumbnailPhotoFileName|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------------+-------------------+----------------+--------------------+----------------------+--------------------+--------------------+
|      680|HL Road Frame - B...|   FR-R92B-58|Black|   1059.3100|1431.5000|  58|1016.04|               18|             6|2002-06-01 00:00:00|               null|            null|[47 49 46 38 39 6...|  no_i

### Prueba con consultas simples usando Pyspark:  
 

#### 1. Listar todas las tablas disponibles

In [0]:
query = "(SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE') AS temp"
df_tables = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)
df_tables.show()


+--------------------+
|          TABLE_NAME|
+--------------------+
|            Customer|
|        ProductModel|
|  ProductDescription|
|             Product|
|ProductModelProdu...|
|     ProductCategory|
|        BuildVersion|
|            ErrorLog|
|             Address|
|     CustomerAddress|
|    SalesOrderDetail|
|    SalesOrderHeader|
+--------------------+



#### 2. Productos con precios mayores a $50

In [0]:
query = "(SELECT ProductID, Name, ListPrice FROM SalesLT.Product WHERE ListPrice > 50) AS temp"
df_filtered = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)
df_filtered.show()


+---------+--------------------+---------+
|ProductID|                Name|ListPrice|
+---------+--------------------+---------+
|      680|HL Road Frame - B...|1431.5000|
|      706|HL Road Frame - R...|1431.5000|
|      717|HL Road Frame - R...|1431.5000|
|      718|HL Road Frame - R...|1431.5000|
|      719|HL Road Frame - R...|1431.5000|
|      720|HL Road Frame - R...|1431.5000|
|      721|HL Road Frame - R...|1431.5000|
|      722|LL Road Frame - B...| 337.2200|
|      723|LL Road Frame - B...| 337.2200|
|      724|LL Road Frame - B...| 337.2200|
|      725|LL Road Frame - R...| 337.2200|
|      726|LL Road Frame - R...| 337.2200|
|      727|LL Road Frame - R...| 337.2200|
|      728|LL Road Frame - R...| 337.2200|
|      729|LL Road Frame - R...| 337.2200|
|      730|LL Road Frame - R...| 337.2200|
|      731|ML Road Frame - R...| 594.8300|
|      732|ML Road Frame - R...| 594.8300|
|      733|ML Road Frame - R...| 594.8300|
|      734|ML Road Frame - R...| 594.8300|
+---------+

#### 3. Contar productos por categoría

In [0]:
query = """
(SELECT ProductCategoryID, COUNT(*) AS TotalProducts
 FROM SalesLT.Product
 GROUP BY ProductCategoryID) AS temp
"""
df_count = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)
df_count.show()


+-----------------+-------------+
|ProductCategoryID|TotalProducts|
+-----------------+-------------+
|                5|           32|
|                6|           43|
|                7|           22|
|                8|            8|
|                9|            3|
|               10|            2|
|               11|            1|
|               12|            3|
|               13|            2|
|               14|            3|
|               15|            3|
|               16|           28|
|               17|            7|
|               18|           33|
|               19|            9|
|               20|           18|
|               21|           14|
|               22|            3|
|               23|            1|
|               24|            6|
+-----------------+-------------+
only showing top 20 rows



Usamos Spark, para realizar un ordenamiento según el total de productos.

In [0]:
query = """
(SELECT ProductCategoryID, COUNT(*) AS TotalProducts
 FROM SalesLT.Product
 GROUP BY ProductCategoryID) AS temp
"""
# Ordenar los resultados en Spark
df_count_sorted = df_count.orderBy("TotalProducts", ascending=False)
df_count_sorted.show()


+-----------------+-------------+
|ProductCategoryID|TotalProducts|
+-----------------+-------------+
|                6|           43|
|               18|           33|
|                5|           32|
|               16|           28|
|                7|           22|
|               20|           18|
|               21|           14|
|               41|           11|
|               19|            9|
|                8|            8|
|               25|            8|
|               17|            7|
|               26|            7|
|               24|            6|
|               27|            4|
|                9|            3|
|               12|            3|
|               14|            3|
|               15|            3|
|               22|            3|
+-----------------+-------------+
only showing top 20 rows



#### 4. Contar el total de productos por tamaño

In [0]:
df.groupBy("Size").count().orderBy("count", ascending=False).show()


+----+-----+
|Size|count|
+----+-----+
|null|    4|
|   M|    2|
|  58|    2|
|   L|    1|
|   S|    1|
+----+-----+



> Mejorar/corregir la query anterior

In [0]:
from pyspark.sql.functions import desc

df.filter(df["Size"].isNotNull()) \
  .groupBy("Size") \
  .count() \
  .withColumnRenamed("count", "TotalCount") \
  .orderBy(desc("TotalCount")) \
  .show()


+----+----------+
|Size|TotalCount|
+----+----------+
|   M|         2|
|  58|         2|
|   L|         1|
|   S|         1|
+----+----------+



Resumen de las mejoras:

- Alias en las columnas: Usamos .withColumnRenamed() para cambiar el nombre de la columna de count a algo más legible como TotalCount.
Uso de desc() para ordenación: 
- Reemplazamos el parámetro ascending=False con la función desc(), que puede ser más clara al leer el código.
- Filtro de valores nulos: Si tu columna "Size" puede tener valores nulos y deseas excluirlos, puedes usar .filter().
- Optimización con cache() y particiones: Si el DataFrame es grande, el uso de cache() o repartition() puede mejorar el rendimiento.

#### 5. Calcular el precio promedio de los productos

In [0]:
df.selectExpr("AVG(ListPrice) AS AveragePrice").show()


+------------+
|AveragePrice|
+------------+
|309.59400000|
+------------+



> ¿y si obtenemos el precio promedio por cada producto?. Mejorar la query

In [0]:
from pyspark.sql.functions import round

df.groupBy("ProductID", "Name") \
  .agg({"ListPrice": "avg"}) \
  .withColumnRenamed("avg(ListPrice)", "AveragePrice") \
  .withColumn("AveragePrice", round("AveragePrice", 2)) \
  .show()



+---------+--------------------+------------+
|ProductID|                Name|AveragePrice|
+---------+--------------------+------------+
|      712|        AWC Logo Cap|        8.99|
|      706|HL Road Frame - R...|     1431.50|
|      713|Long-Sleeve Logo ...|       49.99|
|      709|Mountain Bike Soc...|        9.50|
|      710|Mountain Bike Soc...|        9.50|
|      708|Sport-100 Helmet,...|       34.99|
|      680|HL Road Frame - B...|     1431.50|
|      711|Sport-100 Helmet,...|       34.99|
|      707|Sport-100 Helmet,...|       34.99|
|      714|Long-Sleeve Logo ...|       49.99|
+---------+--------------------+------------+



#### 6. Encontrar productos sin categoría asignada

In [0]:
df.filter(df.ProductCategoryID.isNull()).select("ProductID", "Name", "ProductCategoryID").show()


+---------+----+-----------------+
|ProductID|Name|ProductCategoryID|
+---------+----+-----------------+
+---------+----+-----------------+



> ¿Es la query anterior correcta? Demostrar con otra query que si

In [0]:
# Verificar si hay registros con ProductCategoryID nulo
count_nulls = df.filter(df.ProductCategoryID.isNull()).count()
print(f"Registros con ProductCategoryID NULL: {count_nulls}")

# Verificar los registros con ProductCategoryID NULL
if count_nulls > 0:
    print("Registros con ProductCategoryID NULL:")
    df.filter(df.ProductCategoryID.isNull()).select("ProductID", "Name", "ProductCategoryID").show()
else:
    print("No hay registros con ProductCategoryID NULL.")

# Verificar si hay registros con ProductCategoryID no nulo
count_non_nulls = df.filter(df.ProductCategoryID.isNotNull()).count()
print(f"Registros con ProductCategoryID NO NULL: {count_non_nulls}")

# Verificar los registros con ProductCategoryID no nulo
if count_non_nulls > 0:
    print("Registros con ProductCategoryID NO NULL:")
    df.filter(df.ProductCategoryID.isNotNull()).select("ProductID", "Name", "ProductCategoryID").show()
else:
    print("No hay registros con ProductCategoryID NO NULL.")


Registros con ProductCategoryID NULL: 0
No hay registros con ProductCategoryID NULL.
Registros con ProductCategoryID NO NULL: 10
Registros con ProductCategoryID NO NULL:
+---------+--------------------+-----------------+
|ProductID|                Name|ProductCategoryID|
+---------+--------------------+-----------------+
|      680|HL Road Frame - B...|               18|
|      706|HL Road Frame - R...|               18|
|      707|Sport-100 Helmet,...|               35|
|      708|Sport-100 Helmet,...|               35|
|      709|Mountain Bike Soc...|               27|
|      710|Mountain Bike Soc...|               27|
|      711|Sport-100 Helmet,...|               35|
|      712|        AWC Logo Cap|               23|
|      713|Long-Sleeve Logo ...|               25|
|      714|Long-Sleeve Logo ...|               25|
+---------+--------------------+-----------------+



#### 7. Contar productos por color

In [0]:
df.groupBy("Color").count().orderBy("count", ascending=False).show()


+-----+-----+
|Color|count|
+-----+-----+
|Multi|    3|
|White|    2|
|Black|    2|
|  Red|    2|
| Blue|    1|
+-----+-----+



#### 8. Calcular el costo total de todos los productos

In [0]:
df.selectExpr("SUM(StandardCost) AS TotalCost").show()


+---------+
|TotalCost|
+---------+
|2248.5784|
+---------+



> Mejorar query anterior

In [0]:
from pyspark.sql.functions import round

df.groupBy("ProductCategoryID").agg({"StandardCost": "sum"}) \
  .withColumnRenamed("sum(StandardCost)", "TotalCost") \
  .withColumn("TotalCost", round("TotalCost", 2)) \
  .orderBy(desc("TotalCost")) \
  .show()


+-----------------+---------+
|ProductCategoryID|TotalCost|
+-----------------+---------+
|               18|  2118.62|
|               25|    76.98|
|               35|    39.26|
|               23|     6.92|
|               27|     6.79|
+-----------------+---------+



#### 9. Productos que contienen una palabra específica en su nombre 

In [0]:
# Filtrar productos que contienen la palabra 'Helmet' en el nombre
df.filter(df.Name.contains("Helmet")).select("ProductID", "Name", "ListPrice").show()


+---------+--------------------+---------+
|ProductID|                Name|ListPrice|
+---------+--------------------+---------+
|      707|Sport-100 Helmet,...|  34.9900|
|      708|Sport-100 Helmet,...|  34.9900|
|      711|Sport-100 Helmet,...|  34.9900|
+---------+--------------------+---------+



> Lista los nombres de todos los productos

In [0]:
df.select("Name").show()

+--------------------+
|                Name|
+--------------------+
|All-Purpose Bike ...|
|        AWC Logo Cap|
|Bike Wash - Disso...|
|          Cable Lock|
|               Chain|
|     Classic Vest, L|
|     Classic Vest, M|
|     Classic Vest, S|
|Fender Set - Moun...|
|        Front Brakes|
+--------------------+



#### 10. Listar productos creados después de 2005

In [0]:
df.filter(df.SellStartDate >= "2005-01-01").select("ProductID", "Name", "SellStartDate").show()


+---------+--------------------+-------------------+
|ProductID|                Name|      SellStartDate|
+---------+--------------------+-------------------+
|      707|Sport-100 Helmet,...|2005-07-01 00:00:00|
|      708|Sport-100 Helmet,...|2005-07-01 00:00:00|
|      709|Mountain Bike Soc...|2005-07-01 00:00:00|
|      710|Mountain Bike Soc...|2005-07-01 00:00:00|
|      711|Sport-100 Helmet,...|2005-07-01 00:00:00|
|      712|        AWC Logo Cap|2005-07-01 00:00:00|
|      713|Long-Sleeve Logo ...|2005-07-01 00:00:00|
|      714|Long-Sleeve Logo ...|2005-07-01 00:00:00|
+---------+--------------------+-------------------+



#### 11. Producto más caro por categoría 

In [0]:
from pyspark.sql.functions import col, max as spark_max

df.groupBy("ProductCategoryID").agg(spark_max("ListPrice").alias("MaxPrice")).orderBy("MaxPrice", ascending=False).show()


+-----------------+---------+
|ProductCategoryID| MaxPrice|
+-----------------+---------+
|               18|1431.5000|
|               25|  49.9900|
|               35|  34.9900|
|               27|   9.5000|
|               23|   8.9900|
+-----------------+---------+



#### 12. Calcular el precio promedio por categoría

In [0]:
from pyspark.sql.functions import avg

df.groupBy("ProductCategoryID").agg(avg("ListPrice").alias("AveragePrice")).orderBy("AveragePrice", ascending=False).show()


+-----------------+-------------+
|ProductCategoryID| AveragePrice|
+-----------------+-------------+
|               18|1431.50000000|
|               25|  49.99000000|
|               35|  34.99000000|
|               27|   9.50000000|
|               23|   8.99000000|
+-----------------+-------------+



> Mejorar query anterior

In [0]:
df.groupBy("ProductCategoryID") \
  .agg(avg("ListPrice").alias("AveragePrice")) \
  .withColumn("AveragePrice", round(col("AveragePrice"), 2)) \
  .filter(col("AveragePrice") > 50) \
  .orderBy(col("AveragePrice").desc()) \
  .show()

+-----------------+------------+
|ProductCategoryID|AveragePrice|
+-----------------+------------+
|               18|     1431.50|
+-----------------+------------+



#### 13. Encontrar productos descontinuados (Discontinued no es NULL)

In [0]:
df.filter(df.DiscontinuedDate.isNotNull()).select("ProductID", "Name", "DiscontinuedDate").show()


+---------+----+----------------+
|ProductID|Name|DiscontinuedDate|
+---------+----+----------------+
+---------+----+----------------+



> Comprueba con otra query que la salida anterior es correcta

#### 14. Productos con precios mayores que su costo estándar 

In [0]:
df.filter(df.ListPrice > df.StandardCost).select("ProductID", "Name", "ListPrice", "StandardCost").show()

+---------+--------------------+---------+------------+
|ProductID|                Name|ListPrice|StandardCost|
+---------+--------------------+---------+------------+
|      680|HL Road Frame - B...|1431.5000|   1059.3100|
|      706|HL Road Frame - R...|1431.5000|   1059.3100|
|      707|Sport-100 Helmet,...|  34.9900|     13.0863|
|      708|Sport-100 Helmet,...|  34.9900|     13.0863|
|      709|Mountain Bike Soc...|   9.5000|      3.3963|
|      710|Mountain Bike Soc...|   9.5000|      3.3963|
|      711|Sport-100 Helmet,...|  34.9900|     13.0863|
|      712|        AWC Logo Cap|   8.9900|      6.9223|
|      713|Long-Sleeve Logo ...|  49.9900|     38.4923|
|      714|Long-Sleeve Logo ...|  49.9900|     38.4923|
+---------+--------------------+---------+------------+



### Actividad 1: Repetir las consultas anteriores pero usando SQL (no pyspark) 

#### Registrar las tablas que utilizarás como tabla temporal, por ejemplo:

In [0]:
query = "(SELECT * FROM SalesLT.Product) AS temp"
df = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)
df.createOrReplaceTempView("Product")


In [0]:
%sql
SELECT ProductID, Name, ListPrice
FROM Product
ORDER BY ListPrice DESC
LIMIT 5;


ProductID,Name,ListPrice
749,"Road-150 Red, 62",3578.2700
753,"Road-150 Red, 56",3578.2700
750,"Road-150 Red, 44",3578.2700
751,"Road-150 Red, 48",3578.2700
752,"Road-150 Red, 52",3578.2700


#### 1. Listar todas las tablas disponibles

In [0]:
%sql
SHOW TABLES;


database,tableName,isTemporary
,product,true


#### 2. Productos con precios mayores a $50

In [0]:
%sql
SELECT ProductID, Name, ListPrice
FROM Product
WHERE ListPrice > 50
LIMIT 5;


ProductID,Name,ListPrice
680,"HL Road Frame - Black, 58",1431.5000
706,"HL Road Frame - Red, 58",1431.5000
717,"HL Road Frame - Red, 62",1431.5000
718,"HL Road Frame - Red, 44",1431.5000
719,"HL Road Frame - Red, 48",1431.5000


#### 3. Contar productos por categoría

In [0]:
%sql
SELECT ProductCategoryID, COUNT(*) AS ProductCount
FROM Product
GROUP BY ProductCategoryID
ORDER BY ProductCount DESC
LIMIT 5;


ProductCategoryID,ProductCount
6,43
18,33
5,32
16,28
7,22


#### 4. Contar el total de productos por tamaño

In [0]:
%sql
SELECT Size, COUNT(*) AS ProductCount
FROM Product
GROUP BY Size
ORDER BY ProductCount
LIMIT 5;


Size,ProductCount
70,1
56,2
XL,3
54,9
S,9


#### 5. Calcular el precio promedio de los productos

In [0]:
%sql
-- Verifica la cantidad de valores no nulos
SELECT COUNT(*) AS TotalRows, COUNT(ListPrice) AS NonNullListPrice, COUNT(*) - COUNT(ListPrice) AS NullListPrice
FROM Product;

-- Filtra valores nulos si es necesario
SELECT AVG(ListPrice) AS AveragePrice
FROM Product
WHERE ListPrice IS NOT NULL;



AveragePrice
744.59522034


#### 6. Encontrar productos sin categoría asignada

In [0]:
%sql
SELECT ProductID, Name, ProductCategoryID
FROM Product
WHERE ProductCategoryID IS NULL;


ProductID,Name,ProductCategoryID


#### 7. Contar productos por color

In [0]:
%sql
SELECT Color, COUNT(*) AS ProductCount
FROM Product
GROUP BY Color
ORDER BY ProductCount DESC
LIMIT 5;


Color,ProductCount
Black,89
null,50
Red,38
Silver,36
Yellow,36


#### 8. Calcular el costo total de todos los productos

In [0]:
%sql
SELECT SUM(StandardCost) AS TotalCost
FROM Product;


TotalCost
129275.0025


#### 9. Productos que contienen una palabra específica en su nombre 

In [0]:
%sql
SELECT ProductID, Name, ListPrice
FROM Product
WHERE Name LIKE '%Helmet%';


ProductID,Name,ListPrice
708,"Sport-100 Helmet, Black",34.9900
711,"Sport-100 Helmet, Blue",34.9900
707,"Sport-100 Helmet, Red",34.9900


#### 10. Listar productos creados después de 2005

In [0]:
%sql
SELECT ProductID, Name, SellStartDate
FROM Product
WHERE SellStartDate >= '2005-01-01'
LIMIT 5;


ProductID,Name,SellStartDate
707,"Sport-100 Helmet, Red",2005-07-01T00:00:00.000+0000
708,"Sport-100 Helmet, Black",2005-07-01T00:00:00.000+0000
709,"Mountain Bike Socks, M",2005-07-01T00:00:00.000+0000
710,"Mountain Bike Socks, L",2005-07-01T00:00:00.000+0000
711,"Sport-100 Helmet, Blue",2005-07-01T00:00:00.000+0000


#### 11. Producto más caro por categoría 

In [0]:
%sql
SELECT ProductCategoryID, MAX(ListPrice) AS MaxPrice
FROM Product
GROUP BY ProductCategoryID
ORDER BY MaxPrice DESC
LIMIT 5;


ProductCategoryID,MaxPrice
6,3578.2700
5,3399.9900
7,2384.0700
18,1431.5000
16,1364.5000


#### 12. Calcular el precio promedio por categoría

In [0]:
%sql
SELECT ProductCategoryID, AVG(ListPrice) AS AveragePrice
FROM Product
GROUP BY ProductCategoryID
ORDER BY AveragePrice DESC
LIMIT 5;


ProductCategoryID,AveragePrice
5,1683.36500000
6,1597.45000000
7,1425.24818182
18,780.04363636
16,678.25357143


#### 13. Encontrar productos descontinuados (Discontinued no es NULL)

In [0]:
%sql
SELECT ProductID, Name, DiscontinuedDate
FROM Product
WHERE DiscontinuedDate IS NOT NULL;


ProductID,Name,DiscontinuedDate


#### 14. Productos con precios mayores que su costo estánda

In [0]:
%sql
SELECT ProductID, Name, ListPrice, StandardCost
FROM Product
WHERE ListPrice > StandardCost
LIMIT 5;


ProductID,Name,ListPrice,StandardCost
680,"HL Road Frame - Black, 58",1431.5000,1059.3100
706,"HL Road Frame - Red, 58",1431.5000,1059.3100
707,"Sport-100 Helmet, Red",34.9900,13.0863
708,"Sport-100 Helmet, Black",34.9900,13.0863
709,"Mountain Bike Socks, M",9.5000,3.3963


### Actividad 2. Utilizando PySpark responda a las siguientes preguntas:

#### 1. Escribe un código para calcular cuántos productos tienen un ListPrice mayor que el precio promedio de todos los productos.

In [0]:
from pyspark.sql.functions import avg, count

# Calcular el precio promedio de todos los productos
avg_price = df.select(avg("ListPrice")).collect()[0][0]

# Filtrar productos cuyo ListPrice sea mayor al precio promedio
df.filter(df.ListPrice > avg_price).count()

Out[239]: 102

#### 2. Filtra todos los productos cuyo nombre comience con la letra "A" y muestra su ProductID, Name y ListPrice.

In [0]:
df.filter(df.Name.startswith("A")).select("ProductID", "Name", "ListPrice").show()


+---------+--------------------+---------+
|ProductID|                Name|ListPrice|
+---------+--------------------+---------+
|      879|All-Purpose Bike ...| 159.0000|
|      712|        AWC Logo Cap|   8.9900|
+---------+--------------------+---------+



#### 3. Calcula la desviación estándar de la columna StandardCost.

In [0]:
from pyspark.sql.functions import stddev

df.select(stddev("StandardCost").alias("StandardCostStdDev")).show()


+------------------+
|StandardCostStdDev|
+------------------+
| 534.8956068198088|
+------------------+



#### 4. Ordena los productos por ListPrice en orden ascendente y muestra los 10 productos más baratos. 

In [0]:
df.select('Name','ListPrice').orderBy("ListPrice").limit(10).show(truncate=False)


+---------------------+---------+
|Name                 |ListPrice|
+---------------------+---------+
|Patch Kit/8 Patches  |2.2900   |
|Road Tire Tube       |3.9900   |
|Water Bottle - 30 oz.|4.9900   |
|Mountain Tire Tube   |4.9900   |
|Touring Tire Tube    |4.9900   |
|Bike Wash - Dissolver|7.9500   |
|AWC Logo Cap         |8.9900   |
|Road Bottle Cage     |8.9900   |
|Racing Socks, M      |8.9900   |
|Racing Socks, L      |8.9900   |
+---------------------+---------+



#### 5. Filtra los productos cuyo tamaño sea "M" y cuyo precio sea mayor a $50. 

In [0]:
df.filter((df.Size == "M") & (df.ListPrice > 50)).select("ProductID", "Name", "ListPrice").show()


+---------+--------------------+---------+
|ProductID|                Name|ListPrice|
+---------+--------------------+---------+
|      849|Men's Sports Shor...|  59.9900|
|      853|   Women's Tights, M|  74.9900|
|      856| Men's Bib-Shorts, M|  89.9900|
|      865|     Classic Vest, M|  63.5000|
|      868|Women's Mountain ...|  69.9900|
|      882|Short-Sleeve Clas...|  53.9900|
+---------+--------------------+---------+



#### 6. Escribe un código para contar los productos donde la columna Color es nula

In [0]:
df.filter(df.Color.isNull()).count()


Out[244]: 50

#### 7. Escribe un código para listar todas las combinaciones únicas de Color y Size en la tabla.

In [0]:
df.select("Color", "Size").distinct().show()


+-----+----+
|Color|Size|
+-----+----+
|  Red|  58|
|  Red|  44|
|Multi|  XL|
|  Red|  62|
|  Red|  52|
|Multi|null|
|Black|  44|
|Multi|   L|
|Black|  62|
|Black|  60|
|White|   L|
|Multi|   M|
|Multi|   S|
|  Red|  56|
|  Red|  60|
| Blue|null|
|Black|  58|
|White|   M|
|Black|null|
|  Red|null|
+-----+----+
only showing top 20 rows



#### 8. Calcula la diferencia promedio entre ListPrice y StandardCost para todos los productos. 

In [0]:
from pyspark.sql.functions import col

df.select((col("ListPrice") - col("StandardCost")).alias("PriceDiff")).agg(avg("PriceDiff").alias("AvgPriceDiff")).show()


+------------+
|AvgPriceDiff|
+------------+
|306.37487288|
+------------+



#### 9. ¿Cuáles son los productos cuya fecha de modificación se encuentra entre el 11 de marzo de 2008 a las 10:01:00 y el 11 de marzo de 2008 a las 10:03:00, mostrando el ProductID, Name y ModifiedDate?

In [0]:
from pyspark.sql.functions import to_timestamp, lit

# Filtrar los productos cuya fecha de modificación esté entre el 11 de marzo de 2008 a las 10:01:00 y el 11 de marzo de 2008 a las 10:03:00
df.filter(
    (df.ModifiedDate >= to_timestamp(lit("2008-03-11 10:01:00"), "yyyy-MM-dd HH:mm:ss")) &
    (df.ModifiedDate <= to_timestamp(lit("2008-03-11 10:03:00"), "yyyy-MM-dd HH:mm:ss"))
).select("ProductID", "Name", "ModifiedDate").show()



+---------+--------------------+--------------------+
|ProductID|                Name|        ModifiedDate|
+---------+--------------------+--------------------+
|      680|HL Road Frame - B...|2008-03-11 10:01:...|
|      706|HL Road Frame - R...|2008-03-11 10:01:...|
|      707|Sport-100 Helmet,...|2008-03-11 10:01:...|
|      708|Sport-100 Helmet,...|2008-03-11 10:01:...|
|      709|Mountain Bike Soc...|2008-03-11 10:01:...|
|      710|Mountain Bike Soc...|2008-03-11 10:01:...|
|      711|Sport-100 Helmet,...|2008-03-11 10:01:...|
|      712|        AWC Logo Cap|2008-03-11 10:01:...|
|      713|Long-Sleeve Logo ...|2008-03-11 10:01:...|
|      714|Long-Sleeve Logo ...|2008-03-11 10:01:...|
|      715|Long-Sleeve Logo ...|2008-03-11 10:01:...|
|      716|Long-Sleeve Logo ...|2008-03-11 10:01:...|
|      717|HL Road Frame - R...|2008-03-11 10:01:...|
|      718|HL Road Frame - R...|2008-03-11 10:01:...|
|      719|HL Road Frame - R...|2008-03-11 10:01:...|
|      720|HL Road Frame - R

#### 10. ¿Cuáles son los productos de cada categoría (ProductCategoryID) que tienen un precio (ListPrice) mayor al costo estándar (StandardCost), y cuántos productos cumplen esta condición por categoría?

In [0]:
df.filter(df.ListPrice > df.StandardCost) \
  .groupBy("ProductCategoryID") \
  .count() \
  .show()

+-----------------+-----+
|ProductCategoryID|count|
+-----------------+-----+
|               31|    1|
|               34|    1|
|               28|    3|
|               27|    4|
|               26|    7|
|               12|    3|
|               22|    3|
|               13|    2|
|               16|   28|
|                6|   43|
|               40|    2|
|               20|   18|
|                5|   32|
|               19|    9|
|               41|   11|
|               15|    3|
|               37|    3|
|               17|    7|
|                9|    3|
|               35|    3|
+-----------------+-----+
only showing top 20 rows



#### 11. Filtra los productos cuyo ListPrice esté entre $20 y $100.

In [0]:
df.filter((df.ListPrice >= 20) & (df.ListPrice <= 100)) \
  .select("ProductID", "Name", "ListPrice") \
  .show()


+---------+--------------------+---------+
|ProductID|                Name|ListPrice|
+---------+--------------------+---------+
|      707|Sport-100 Helmet,...|  34.9900|
|      708|Sport-100 Helmet,...|  34.9900|
|      711|Sport-100 Helmet,...|  34.9900|
|      713|Long-Sleeve Logo ...|  49.9900|
|      714|Long-Sleeve Logo ...|  49.9900|
|      715|Long-Sleeve Logo ...|  49.9900|
|      716|Long-Sleeve Logo ...|  49.9900|
|      805|          LL Headset|  34.2000|
|      808|LL Mountain Handl...|  44.5400|
|      809|ML Mountain Handl...|  61.9200|
|      811|  LL Road Handlebars|  44.5400|
|      812|  ML Road Handlebars|  61.9200|
|      815|LL Mountain Front...|  60.7450|
|      818| LL Road Front Wheel|  85.5650|
|      823|LL Mountain Rear ...|  87.7450|
|      841|Men's Sports Shor...|  59.9900|
|      843|          Cable Lock|  25.0000|
|      845|       Mountain Pump|  24.9900|
|      847|Headlights - Dual...|  34.9900|
|      848|Headlights - Weat...|  44.9900|
+---------+

#### 12. Para cada ProductCategoryID, muestra los 5 productos con el costo (StandardCost) más alto.

In [0]:
from pyspark.sql import functions as F

# Primero, obtenemos los productos con el costo más alto por categoría
top_products = df.groupBy("ProductCategoryID", "ProductID") \
    .agg(F.max("StandardCost").alias("MaxStandardCost")) \
    .withColumn("rank", F.rank().over(Window.partitionBy("ProductCategoryID").orderBy(F.desc("MaxStandardCost"))))

# Filtramos los productos que tienen el rango 1, es decir, los 5 primeros productos con el costo más alto
top_5_products = top_products.filter(top_products.rank <= 5)

# Hacemos un join con la tabla original para obtener los detalles de esos productos
result = top_5_products.join(df, on=["ProductCategoryID", "ProductID"], how="inner") \
    .select("ProductCategoryID", "ProductID", "Name", "StandardCost")

result.show()


+-----------------+---------+--------------------+------------+
|ProductCategoryID|ProductID|                Name|StandardCost|
+-----------------+---------+--------------------+------------+
|                5|      774|Mountain-100 Silv...|   1912.1544|
|                5|      773|Mountain-100 Silv...|   1912.1544|
|                5|      771|Mountain-100 Silv...|   1912.1544|
|                5|      772|Mountain-100 Silv...|   1912.1544|
|                5|      776|Mountain-100 Blac...|   1898.0944|
|                5|      778|Mountain-100 Blac...|   1898.0944|
|                5|      775|Mountain-100 Blac...|   1898.0944|
|                5|      777|Mountain-100 Blac...|   1898.0944|
|                6|      750|    Road-150 Red, 44|   2171.2942|
|                6|      751|    Road-150 Red, 48|   2171.2942|
|                6|      752|    Road-150 Red, 52|   2171.2942|
|                6|      753|    Road-150 Red, 56|   2171.2942|
|                6|      749|    Road-15

#### 13. Filtra los productos cuya columna ThumbNailPhoto no es nula y muestra su ProductID, Name, y ThumbNailPhotoFileName.

In [0]:
df.filter(df.ThumbNailPhoto.isNotNull()) \
  .select("ProductID", "Name", "ThumbNailPhotoFileName") \
  .show()


+---------+--------------------+----------------------+
|ProductID|                Name|ThumbNailPhotoFileName|
+---------+--------------------+----------------------+
|      680|HL Road Frame - B...|  no_image_availabl...|
|      706|HL Road Frame - R...|  no_image_availabl...|
|      707|Sport-100 Helmet,...|  no_image_availabl...|
|      708|Sport-100 Helmet,...|  no_image_availabl...|
|      709|Mountain Bike Soc...|  no_image_availabl...|
|      710|Mountain Bike Soc...|  no_image_availabl...|
|      711|Sport-100 Helmet,...|  no_image_availabl...|
|      712|        AWC Logo Cap|  no_image_availabl...|
|      713|Long-Sleeve Logo ...|  awc_jersey_male_s...|
|      714|Long-Sleeve Logo ...|  awc_jersey_male_s...|
|      715|Long-Sleeve Logo ...|  awc_jersey_male_s...|
|      716|Long-Sleeve Logo ...|  awc_jersey_male_s...|
|      717|HL Road Frame - R...|  no_image_availabl...|
|      718|HL Road Frame - R...|  no_image_availabl...|
|      719|HL Road Frame - R...|  no_image_avail

#### 14. Agrupa los productos por ProductCategoryID y calcula el precio total (SUM(ListPrice)) por categoría. 

In [0]:
from pyspark.sql.functions import sum

df.groupBy("ProductCategoryID") \
  .agg(sum("ListPrice").alias("TotalPrice")) \
  .show()


+-----------------+----------+
|ProductCategoryID|TotalPrice|
+-----------------+----------+
|               31|  159.0000|
|               34|   21.9800|
|               28|  224.9700|
|               27|   36.9800|
|               26|  449.9300|
|               12|  836.9700|
|               22|  269.9700|
|               13|  212.9500|
|               16|18991.1000|
|                6|68690.3500|
|               40|   44.9800|
|               20|11365.4800|
|                5|53867.6800|
|               19|  356.7000|
|               41|  214.3100|
|               15|  261.2200|
|               37|   93.9700|
|               17|  448.1300|
|                9|  276.7200|
|               35|  104.9700|
+-----------------+----------+
only showing top 20 rows



#### 15. Filtra los productos cuya fecha de inicio de venta (SellStartDate) esté entre el 1 de enero de 2005 y el 31 de diciembre de 2006. 

In [0]:
df.filter((df.SellStartDate >= "2005-01-01") & (df.SellStartDate <= "2006-12-31")) \
  .select("ProductID", "Name", "SellStartDate") \
  .show()


+---------+--------------------+-------------------+
|ProductID|                Name|      SellStartDate|
+---------+--------------------+-------------------+
|      707|Sport-100 Helmet,...|2005-07-01 00:00:00|
|      708|Sport-100 Helmet,...|2005-07-01 00:00:00|
|      709|Mountain Bike Soc...|2005-07-01 00:00:00|
|      710|Mountain Bike Soc...|2005-07-01 00:00:00|
|      711|Sport-100 Helmet,...|2005-07-01 00:00:00|
|      712|        AWC Logo Cap|2005-07-01 00:00:00|
|      713|Long-Sleeve Logo ...|2005-07-01 00:00:00|
|      714|Long-Sleeve Logo ...|2005-07-01 00:00:00|
|      715|Long-Sleeve Logo ...|2005-07-01 00:00:00|
|      716|Long-Sleeve Logo ...|2005-07-01 00:00:00|
|      717|HL Road Frame - R...|2005-07-01 00:00:00|
|      718|HL Road Frame - R...|2005-07-01 00:00:00|
|      719|HL Road Frame - R...|2005-07-01 00:00:00|
|      720|HL Road Frame - R...|2005-07-01 00:00:00|
|      721|HL Road Frame - R...|2005-07-01 00:00:00|
|      722|LL Road Frame - B...|2005-07-01 00:

### Actividad 3.  
 La Empresa decide "migrar" de sql server a postgreSQL. Efectuar la conexion Databricks Community con PostgreSQL en Azure. Efectuar algunas consultas sobre PostgreSQL usando PySpark y Scala. Utiliza una base de datos cualquiera.  
Si la version de community da muchos problemas utilizar Azure Databricks.